In [1127]:
from random import *
import hashlib

def puissance (a,e,n):
  p = 1
  while e>0:
    if (e % 2 != 0):
      p=(p*a)%n
    a=(a*a)%n
    e=e//2
  return p

In [1128]:
# Test puissance
def test_puissance():
    assert(puissance(4,2,5)==1)
test_puissance()

In [1129]:
def test_premier_Fermat(n): # test de Fermat
   if ( (puissance(2,n-1,n)==1) and
      (puissance(3,n-1,n)==1) and
      (puissance(5,n-1,n)==1) and
      (puissance(7,n-1,n)==1) and
      (puissance(11,n-1,n)==1) and
      (puissance(13,n-1,n)==1) ):
      return True; # probablement premier (garantie si n<2^15)
   return False

In [1130]:
# Test premier de Fermat
def test_Fermat():
    assert(test_premier_Fermat(22)==False)
    assert(test_premier_Fermat(647)==True)
test_Fermat()

In [1131]:
def temoin_Miller(n, a):
    s = 0
    d = n-1
    while d % 2 == 0:
       d = d//2
    x = puissance(a, d, n)
    if(x == 1 or x == n-1):
        return False
    for i in range(s-1):
        x = puissance(x, 2, n)
        if(x == n-1):
            return False
    return True

In [1132]:
# Test témoin de Miller 
def test_temoin_Miller():
    assert(temoin_Miller(3,9)==True)
    assert(temoin_Miller(4,9)==False)
test_temoin_Miller()

In [1133]:
# Le GMP recommande k=25 pour avoir une proba d'erreur < 2**-50
def test_premier_Miller_Rabin(n, k):
    if n == 2 or n == 3:
       return True
    if n % 2 == 0:
       return False
    for i in range(k):
        a = randint(2, n-2)
        if(temoin_Miller(n, a)):
            return False
    return True

In [1134]:
# Test de primalité de Miller Rabin
def test_premier_Miller():
    assert(test_premier_Miller_Rabin(3,1)==True)
    assert(test_premier_Miller_Rabin(42,5)==False)
test_premier_Miller()

In [1135]:
def pgcd(u, v):
  t = 0
  while (v):
    t = u
    u = v
    v = t % v
  if(u<0):
    return -u
  else:
    return u

In [1136]:
# Test PGCD
def test_pgcd():
    assert(pgcd(5,4)==1)
    assert(pgcd(222,448)==2)
test_pgcd()

In [1137]:
def bezout(a, b):

  # On sauvegarde les valeurs de a et b.
   a0 = a
   b0 = b

  # On laisse invariant p*a0 + q*b0 = a et  r*a0 + s*b0 = b.
   p = 1
   q = 0
   r = 0
   s = 1
   c = 1
   quotient = 1
   nouveau_r = 1
   nouveau_s = 1

  # La boucle principale.
   while (b != 0):
    c = a % b
    quotient = a//b
    a = b
    b = c
    nouveau_r = p - (quotient * r)
    nouveau_s = q - (quotient * s)
    p = r
    q = s
    r = nouveau_r
    s = nouveau_s
   if p < 0: # Si p est négatif retourne p+phi
     p = p+b0
   return p  # on n'a besoin que de p 

In [1138]:
# Test théorème de Bezout
def test_Bezout():
    assert(bezout(50,47)==16)
    assert(bezout(15,5)==0)
    assert(bezout(445,457)==38)
test_Bezout()

In [1139]:
def generateurCle(c,b):
  p=randint(c,b)
  q=randint(c,b)
  e=randint(c,b)
  d=randint(c,b)

  while test_premier_Miller_Rabin(p,25)==False:
    p = randint(c,b)
  while test_premier_Miller_Rabin(q,25)==False:
    q = randint(c,b)

  n=p*q
  
  phi = (p - 1)*(q - 1)
 
  while 1>=e>=phi or pgcd(e,phi)!=1 :
      e = randint(c,b)
  d= int(bezout(int(e),int(phi)))
  return e,d,n 

In [1140]:
def chiffrer(message,clé,n):
    return puissance(message,clé,n)

In [1141]:
# Test de la fonction chiffrer
def test_chiffrer():
    assert(chiffrer(1234,41,2449)==1927)
test_chiffrer()

In [1142]:
def dechiffrer(messageChiffre,clé,n):
    return puissance(messageChiffre,clé,n)

In [1143]:
# Test de la fonction dechiffrer 
def test_dechiffrer():
    assert(dechiffrer(1927,1541,2449)==1234)
test_dechiffrer()

In [1144]:
def str_to_int(m):
    s=0
    b=1
    for i in range(len(m)):
        s=s+ord(m[i])*b
        b=b*256
    return s

In [1145]:
def test_str_to_int():
    assert(str_to_int('a')==97)
    assert(str_to_int('k')==107)
test_str_to_int()

In [1146]:
def int_to_str(c):
    s =""
    q,r= divmod(c,256)
    s=s+str(chr(r))
    while q != 0:
        q,r = divmod(q,256)
        s =s +str(chr(r))
    return s 


In [1147]:
def test_int_to_str():
    assert(int_to_str(97)=='a')
    assert(int_to_str(107)=='k')
test_int_to_str()

In [1148]:
def calculEmpreinte(message): 
    m=hashlib.sha256(repr(message).encode())
    return m.hexdigest()

In [1149]:
def decoupe(variable,taille,liste,strVariable):
    decoupeIndex = 0
    for i in range(1,len(variable)):
        if(variable[i]!='0' and i%taille==0):
            liste.append(int(variable[decoupeIndex:i]))
            strVariable+=str(int(variable[decoupeIndex:i]))
            decoupeIndex=i
            
        if(i==len(variable)-1):
            liste.append(int(variable[decoupeIndex:len(variable)]))
            strVariable+=str(int(variable[decoupeIndex:len(variable)]))

In [1150]:
# Test de la fonction de découpe d'un message 
def test_decoupe():
    l=[]
    strL = ''
    liste=[]
    strListe = '123456'
    liste.append(12)
    liste.append(34)
    liste.append(56)
    decoupe(strListe,2,l,strL)
    assert(l== liste)
    
test_decoupe()

In [1151]:
def chiffrerEnsemble(liste, cle):
       for i in range(len(liste)):
            liste[i]=chiffrer(liste[i],cle[0],cle[1])

In [1152]:
# Test de la fonction chiffrer ensemble 
def test_chiffrerEnsemble():
    liste = []
    liste.append(12)
    liste.append(34)
    liste.append(56)
    gC=generateurCle(10**1,10**2)
    clé= [gC[0],gC[2]]
    chiffrerEnsemble(liste,clé)
    l =[]
    l.append(chiffrer(12,clé[0],clé[1]))
    assert(l==liste)

In [1153]:
def chiffrement_Asymetrique_Signe(message, clePrivee, clePublique):
    strEmpreinte = ""
    strEmpreinteC = ""
    messageEmpreinte = []
    messageParti = []
    strMessage = ""

    #Verifie si le message est un string, si oui le convertit en int
    if(isinstance(message, str)):
        message = str_to_int(message)

    #Calcule de l'empreinte du message
    empreinte = str(str_to_int(calculEmpreinte(message)))
    tailleMessage = int(len(str(message))/(len(str(message))/2))
    tailleEmpreinte = int(len(empreinte)/(len(empreinte)/2))

    #Découpe de l'empreinte pour pouvoir mieux la chiffrer
    decoupe(empreinte, tailleEmpreinte, messageEmpreinte, strEmpreinte)
    print("Empreinte: "+empreinte)

    #On decoupe le message en plusieurs morceaux
    decoupe(str(message), tailleMessage, messageParti, strMessage)

    #Chiffrage de l'empreinte avec la clé privée de l'envoyeur
    for i in range(0, len(messageEmpreinte)):
        messageEmpreinte[i] = chiffrer(
            messageEmpreinte[i], clePrivee[0], clePrivee[1])
        strEmpreinteC += str(messageEmpreinte[i])
    print("Empreinte Chiffré: "+strEmpreinteC)

    #Concaténation du message et de l'empreinte chiffrée
    print("Message + empreinte: "+strMessage+strEmpreinteC)

    #On chiffre l'ensemble des empreintes et des messages
    chiffrerEnsemble(messageEmpreinte, clePublique)
    chiffrerEnsemble(messageParti, clePublique)

    #Envoie du message finale chiffrée avec la clé publique du receveur
    return messageParti, messageEmpreinte



In [1154]:
def verif_signature(messageParti, messageEmpreinte, clePrivee, clePublique):
    messageEmpreinteDecryptage = messageEmpreinte
    messagePartiDecryptage = messageParti
    empreinte1 = ""
    empreinteDecrypt = ""
    message = ""

    #Dechiffrage de l'empreinte
    for i in range(len(messageEmpreinteDecryptage)):
      messageEmpreinteDecryptage[i] = dechiffrer(
          messageEmpreinteDecryptage[i], clePrivee[0], clePrivee[1])
      empreinte1 += str(messageEmpreinteDecryptage[i])

    #Dechiffrage du message
    for i in range(len(messagePartiDecryptage)):
      messagePartiDecryptage[i] = dechiffrer(
          messagePartiDecryptage[i], clePrivee[0], clePrivee[1])
      message += str(messagePartiDecryptage[i])

    print("Message déchiffré: "+message+empreinte1)
    print("Message : "+message)
    print("Empreinte du message chiffrée: "+empreinte1)

    #Déchiffrage de l'empreinte avec la clé publique
    for i in range(len(messageEmpreinteDecryptage)):
      messageEmpreinteDecryptage[i] = str(dechiffrer(
          messageEmpreinteDecryptage[i], clePublique[0], clePublique[1]))
      empreinteDecrypt += str(messageEmpreinteDecryptage[i])

    #Vérification de l'empreinte déchiffrer, il faut qu'elle corresponde à l'empreinte du message
    if(str_to_int(calculEmpreinte(int(message)))) == (int(empreinteDecrypt)):
       return int(message)
    else:
       return -1
    

In [1155]:
# Test de l'envoi et de la reception d'un message
def test_envoiEtReceptionMessage():
    message = 1234
    gA = generateurCle(10**38, 10**39)
    clePublique = [gA[0], gA[2]]
    clePrivee = [gA[1], gA[2]]
    messageFinal = chiffrement_Asymetrique_Signe(
        message, clePrivee, clePublique)
    messageVerif = verif_signature(
        messageFinal[0], messageFinal[1], clePrivee, clePublique)
    assert(messageVerif == message)


test_envoiEtReceptionMessage()


Empreinte: 2744372168965420703255761072842808216329619802047789678765124104220163271391731439411469307863996541660886798191291340339988553483486636194939326959792944
Empreinte Chiffré: 17142128397806648628532604485991123953249802123725359430547892070871812813037614168194432263262897567066611167312189959850527162554376125349236090954704340865395652848234671610099391326134044048250432787419802049102714741462432853897666779389140549579112009685734452619231250014074275900995686240280034410812377405069852147663236395823771237990843297253419094182292990629490671468661767553862918236755607422572418040142989680489105874350289567486727128707194629370158593959859793626967977896995808862639736587425313842268956062973897085429866499140015900994036355271116037922982891554346679952312142008104595067115589016700823525431697706076295462691169774908964103847427167119644876032704465474915587230909783813266985939020894008062968268471761062391108707783886109944349732272736071819305132447777874731828034795

In [1156]:
#Alice
gA = generateurCle(10**38, 10**39)
clePubliqueA = [gA[0], gA[2]]
clePriveeA = [gA[1], gA[2]]
print("Clé Publique A: "+str(clePubliqueA))
print("Clé Privée A: "+str(clePriveeA))


Clé Publique A: [515888217598621828669374689300839381513, 376931083058656218163385630690565592290531334567748812582779908038389843042133]
Clé Privée A: [223064989657509013352993944513578860164102185617041200233005554813036957860777, 376931083058656218163385630690565592290531334567748812582779908038389843042133]


In [1157]:
#CA
gCA = generateurCle(10**56, 10**57)
clePubliqueCA = [gCA[0], gCA[2]]
clePriveeCA = [gCA[1], gCA[2]]
print("Clé Publique CA: "+str(clePubliqueCA))
print("Clé Privée CA: "+str(clePriveeCA))


Clé Publique CA: [976421489255677673011355108063360558608194406683356751509, 913239908636416325969121042530315914090509039199144611679579820183071330032852393408164739683428431651811730901533]
Clé Privée CA: [53066256265816023772409596201859939093549601484308711274072313202151782484719381196927709763238160903894781472253, 913239908636416325969121042530315914090509039199144611679579820183071330032852393408164739683428431651811730901533]


In [1158]:
#Envoie le message au CA
messageA=chiffrement_Asymetrique_Signe(int(clePubliqueA[0]),clePriveeA,clePubliqueCA)

Empreinte: 2735201714726831019001997597178423746065900884539712187486828090340197157500539059998642537863451606874398031238518240724958393946213807290044475342403893
Empreinte Chiffré: 29730662087179626976894012682037510520815183395315229163275364152667212680478228898891295689533906136898456389425082546520646533369863069552193397681235981718799409063252449698025529138947685742162413099203224515050798674701364833783317592679816494689734369579311876130049065163801951720587942351218546763019826633800092257659352425036990917333628164539292504643879461096719397724650827147125104937216619649138473039510801863378381438848297489241899172394130151719488712244651393612493576527804900991687552348416435085449119020038001580494675546832782749176715994772814960935970321478670756182322809410405118291976886364421919230676934093213763391075610896940817824659870950000535006605587239313157169131983296019503686178570056688089274465994941232940245801944658640630347598565726291505598168799207778325256731249

In [1159]:
#Vérifie la signature
if(int(verif_signature(messageA[0], messageA[1], clePriveeCA, clePubliqueA)) == clePubliqueA[0]):
    CAClePubliqeA = chiffrer(clePubliqueA[0], clePriveeCA[0], clePriveeCA[1])
    print("Certificat de Alice: "+str(CAClePubliqeA))
else:
    print("Le message a été modifié")


Message déchiffré: 515888217598621828669374689300839381513297306620871796269768940126820375105208151833953152291632753641526672126804782288988912956895339061368984563894250825465206465333698630695521933976812359817187994090632524496980255291389476857421624130992032245150507986747013648337833175926798164946897343695793118761300490651638019517205879423512185467630198266338000922576593524250369909173336281645392925046438794610967193977246508271471251049372166196491384730395108018633783814388482974892418991723941301517194887122446513936124935765278049009916875523484164350854491190200380015804946755468327827491767159947728149609359703214786707561823228094104051182919768863644219192306769340932137633910756108969408178246598709500005350066055872393131571691319832960195036861785700566880892744659949412329402458019446586406303475985657262915055981687992077783252567312491411086486023010908988833645520382307204587214212538460069684093126079573129849230626409041644728302375961752790624887261175926

In [1160]:
#Bob
gB = generateurCle(10**39, 10**40)
clePubliqueB = [gB[0], gB[2]]
clePriveeB = [gB[1], gB[2]]

print("Clé Publique de Bob: "+str(clePubliqueB))
print("Clé Privée de Bob: "+str(clePriveeB))

Clé Publique de Bob: [3719503193403005318519062685506629548359, 81887722705703156918541534820617086594165479853620905068029542425265911978382473]
Clé Privée de Bob: [3601160476347020222637028582327245964437072122167096479138896324379541002772831, 81887722705703156918541534820617086594165479853620905068029542425265911978382473]


In [1161]:
#Bob vérifie le certificat d'Alice
print("Vérification de la clé publique Alice par Bob")
verifCA=dechiffrer(CAClePubliqeA,clePubliqueCA[0],clePubliqueCA[1])
if(verifCA==clePubliqueA[0]):
    print("La clé publique d'Alice est valide: "+str(verifCA))
else:
    print("Le message a été modifié")

Vérification de la clé publique Alice par Bob
La clé publique d'Alice est valide: 515888217598621828669374689300839381513


In [1162]:
# Le message envoyé par Alice
# Le message peut être modifier mais seulement par un autre Integer
messagez = 1435263726352725355365241625326643625185635625635624365265736553
print("Le message d'origine de Alice est "+str(messagez))

# Alice chiffre son message
messageCd = chiffrement_Asymetrique_Signe(messagez, clePriveeA, clePubliqueB)

# Bob déchiffre le message d'Alice et vérifie s'il n'y a pas eu de modification
messageDk = verif_signature(
    messageCd[0], messageCd[1], clePriveeB, clePubliqueA)

if(messageDk == -1):
    print("Le message a été modifier")
else:
    print("Le message décrypté est "+str(messageDk))


Le message d'origine de Alice est 1435263726352725355365241625326643625185635625635624365265736553
Empreinte: 5300891185829031205796577270218220459108867512432011240511409409901811464693004696013459641053381470184764448586881136061539631725538530056516827166946352
Empreinte Chiffré: 12701775548088980833877929481312663222808012930209889362978106082715518492884214660941434309595478173920113794906820035754871047770412269069620965579267016471258049265831610140848711918424366450529723841871021529033086214974791200601129211040007417527490464121860397218848345563513171953535652315163532777211853205526652786083096786621238167309287917539361159696783447938702501388836342743612649730431156640475368761160897768498843762653415772998825932135523981862233621055663219426627498378636205518161816686203335069121410041564594917320955761879940906325244969802552913894768574216241309920322451505079867470136483378333321967003189222559395552657055848430115975977881559880665608143371522568447662304310061956559